In [14]:
import datetime
import json

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
top_500 = pd.read_csv('TOP500_202206.csv')
top_500.head()

,Rank,Previous Rank,First Appearance,First Rank,Name,Computer,Site,Manufacturer,Country,Year,...,Accelerator/Co-Processor,Cores per Socket,Processor Generation,System Model,System Family,Interconnect Family,Interconnect,Continent,Site ID,System ID
0,1,NaN,59,1,Frontier,"HPE Cray EX235a, AMD Optimized 3rd Generation ...",DOE/SC/Oak Ridge National Laboratory,HPE,United States,2021,...,AMD Instinct MI250X,64,AMD Milan,HPE Cray EX235a,HPE Cray EX,Gigabit Ethernet,Slingshot-11,North America,"48,553",180047
1,2,1.0,55,1,Supercomputer Fugaku,"Supercomputer Fugaku, A64FX 48C 2.2GHz, Tofu i...",RIKEN Center for Computational Science,Fujitsu,Japan,2020,...,None,48,Fujitsu A64FX,Supercomputer Fugaku,Fujitsu Cluster,Proprietary Network,Tofu interconnect D,Asia,"50,831",179807
2,3,NaN,59,3,LUMI,"HPE Cray EX235a, AMD Optimized 3rd Generation ...",EuroHPC/CSC,HPE,Finland,2022,...,AMD Instinct MI250X,64,AMD Milan,HPE Cray EX235a,HPE Cray EX,Gigabit Ethernet,Slingshot-11,Europe,"50,908",180048
3,4,2.0,51,1,Summit,"IBM Power System AC922, IBM POWER9 22C 3.07GHz...",DOE/SC/Oak Ridge National Laboratory,IBM,United States,2018,...,NVIDIA Volta GV100,22,IBM POWER9,IBM Power System AC922,IBM Power Systems,Infiniband,Dual-rail Mellanox EDR Infiniband,North America,"48,553",179397
4,5,3.0,51,3,Sierra,"IBM Power System AC922, IBM POWER9 22C 3.1GHz,...",DOE/NNSA/LLNL,IBM / NVIDIA / Mellanox,United States,2018,...,NVIDIA Volta GV100,22,IBM POWER9,IBM Power System AC922,IBM Power Systems,Infiniband,Dual-rail Mellanox EDR Infiniband,North America,"49,763",179398


In [7]:
# top_500.columns
# top_500[['Rank', 'Rmax [TFlop/s]']]

In [10]:
with open('top500.json', 'r') as f:
    history = json.load(f)

In [24]:
start_date = datetime.date(1993, 6, 1)

In [25]:
history_lst = []
for elem in history:
    month, year = [int(part) for part in elem.split('/')]
    cur_date = datetime.date(year, month, 1)
    day_from_start = (cur_date - start_date).days
    top1 = int(history[elem]['#1'][0])
    res = {'day_from_start': day_from_start, 'top1': top1, 'target': np.log(top1)}
    history_lst.append(res)


In [26]:
history_df = pd.DataFrame(history_lst)

In [29]:
history_df.tail()

,day_from_start,top1,target
54,9862,415500000,19.844993
55,10015,442000000,19.906820
56,10227,442000000,19.906820
57,10380,442000000,19.906820
58,10592,1100000000,20.818576


In [31]:
lr = LinearRegression()
lr.fit(np.array(history_df['day_from_start']).reshape(-1, 1), history_df['target'])

LinearRegression()

In [33]:
finish = datetime.date(2025, 5, 1)
preds = np.exp(lr.predict(np.array((finish - start_date).days).reshape(-1, 1)))
preds

array([1.07988915e+10])

Оценка доли затрачиваемой энергии

In [42]:
world_power = 28214.07  # https://ourworldindata.org/grapher/electricity-prod-source-stacked
top_500['Power (kW)'] = top_500['Power (kW)'].str.replace(',', '').astype(float)
top_500['Power (kW)'] = top_500['Power (kW)'].interpolate()
top_500_power = (top_500['Power (kW)'].sum() * 8760) / 1e9
top_500_power / world_power * 100


0.022994592257692705